In [7]:
:dep polars = { version = "0.35.4", features = ["lazy", "describe"] }
:dep polars = { version = "0.35.4", features = ["lazy", "describe"] }
:dep ndarray = "0.15"
:dep ndarray-linalg = { version = "0.16", features = ["openblas-system"] }

// Now we can use these crates
use polars::prelude::*;
use ndarray::prelude::*;
// ... rest of the code

In [14]:
let df = CsvReadOptions::default()
    .try_into_reader_with_file_path(Some("/data/prices.csv".into()))?
    .finish()?;

Error: failed to resolve: use of undeclared type `CsvReadOptions`

In [12]:
// Read the CSV file
let df = CsvReader::from_path("/data/prices.csv")
    .unwrap()
    .infer_schema(None)
    .has_header(true)
    .finish()
    .unwrap();

// Display basic information about the dataframe
println!("DataFrame Shape: {:?}", df.shape());
println!("\nColumn Names:");
println!("{:?}", df.get_column_names());

// Show the first few rows
println!("\nFirst few rows:");
println!("{}", df.head(Some(5)));

// Get basic statistics for the 'close' column
let close_stats = df.column("close")
    .unwrap()
    .describe();
println!("\nClose price statistics:");
println!("{}", close_stats);

// Check for any missing values
let null_counts = df.null_count();
println!("\nNull counts per column:");
println!("{}", null_counts);

// Get unique tickers
let unique_tickers = df.column("ticker")
    .unwrap()
    .unique()
    .unwrap();
println!("\nUnique tickers ({} total):", unique_tickers.len());
println!("{}", unique_tickers);

Error: no method named `describe` found for reference `&polars::prelude::Series` in the current scope

In [11]:

/// Converts wide-format price data to log returns
fn calculate_log_returns(df: &DataFrame) -> Result<DataFrame, PolarsError> {
    // Assuming df is already in wide format with date index and asset columns
    let mut return_df = df.clone();
    
    for col in df.get_columns() {
        if col.name() != "date" {
            let returns = col.cast(&DataType::Float64)?
                .apply(|x| x.ln())  // Using apply for natural log
                .shift(1)
                .sub(&col.cast(&DataType::Float64)?
                    .apply(|x| x.ln()))?
                .mul_f64(-1.0)  // Make it current/previous instead of previous/current
                .rename(col.name());
            return_df.with_column(returns)?;
        }
    }
    
    Ok(return_df.drop_nulls(None)?)
}

/// Standardizes returns (zero mean, unit variance) for each asset
fn standardize_returns(df: &DataFrame) -> Result<DataFrame, PolarsError> {
    let mut std_df = df.clone();
    
    for col in df.get_columns() {
        if col.name() != "date" {
            let mean = col.mean().ok_or_else(|| PolarsError::ComputeError("Mean calculation failed".into()))?;
            let std = col.std(1).ok_or_else(|| PolarsError::ComputeError("Std calculation failed".into()))?;
            
            let standardized = col.sub_f64(mean)?
                .div_f64(std)?
                .rename(col.name());
            std_df.with_column(standardized)?;
        }
    }
    
    Ok(std_df)
}

/// Computes the correlation matrix of returns
fn compute_correlation_matrix(df: &DataFrame) -> Result<DataFrame, PolarsError> {
    // Remove date column if present
    let data_cols = df.get_columns().into_iter()
        .filter(|col| col.name() != "date")
        .collect::<Vec<_>>();
    
    let corr_df = DataFrame::new(data_cols)?
        .corr(None)?;  // Added None parameter for default method
    
    Ok(corr_df)
}

/// Creates rolling windows of returns for time-varying analysis
fn create_rolling_windows(
    df: &DataFrame, 
    window_size: i64,
    step_size: i64
) -> Result<Vec<DataFrame>, PolarsError> {
    let total_rows = df.height() as i64;
    let mut windows = Vec::new();
    
    for start in (0..total_rows - window_size).step_by(step_size as usize) {
        let window = df.slice(start, window_size as usize);
        windows.push(window);
    }
    
    Ok(windows)
}

/// Computes basic return statistics for each asset
fn compute_return_statistics(df: &DataFrame) -> Result<DataFrame, PolarsError> {
    let mut stats = Vec::new();
    
    for col in df.get_columns() {
        if col.name() != "date" {
            let mean = col.mean().ok_or_else(|| PolarsError::ComputeError("Mean calculation failed".into()))?;
            let std = col.std(1).ok_or_else(|| PolarsError::ComputeError("Std calculation failed".into()))?;
            let skew = col.skew().ok_or_else(|| PolarsError::ComputeError("Skew calculation failed".into()))?;
            let kurt = col.kurtosis().ok_or_else(|| PolarsError::ComputeError("Kurtosis

Error: unterminated double quote string

In [ ]:

// Example usage:
fn main() -> Result<(), PolarsError> {
    // Load data (assuming we have it in wide format, if not we'll need a pivot function)
    let df = CsvReader::from_path("/data/prices.csv")?
        .infer_schema(None)
        .has_header(true)
        .finish()?;
    
    // Calculate log returns
    let returns_df = calculate_log_returns(&df)?;
    
    // Get basic statistics
    let stats_df = compute_return_statistics(&returns_df)?;
    println!("Return Statistics:\n{}", stats_df);
    
    // Look at correlations
    let corr_df = compute_correlation_matrix(&returns_df)?;
    println!("Correlation Matrix:\n{}", corr_df);
    
    // Create 252-day rolling windows with 21-day steps (approximately yearly windows, monthly steps)
    let windows = create_rolling_windows(&returns_df, 252, 21)?;
    println!("Number of windows: {}", windows.len());
    
    Ok(())
}